In [9]:
import pandas as pd
import numpy as np
import requests
from  data.pathsandmore import pathname, api_secret, api_id,app_Token
from geopy.geocoders import Nominatim
import pickle
import playsound
import os
import re
import time

from datetime import datetime 
from datetime import timedelta


user_agent="NYCAccidentsProject"
geolocator = Nominatim(user_agent=user_agent)

counties_nyc_dict={'Queens County': 'Queens',
                    'Kings County': 'Brooklyn',
                  'New York County': 'Manhattan',
                  'Bronx County': 'Bronx',
                  'Richmond County': 'Staten Island'}
boroughs=list(counties_nyc_dict.values())



def play_done_message():
    playsound.playsound("./data/JobDone.m4a")

def adding_boro(row):
    if pd.isna(row['borough']) and row['address']=='Nothing':
        #print(row['address'])
        return np.nan
    elif pd.isna(row['borough']) and not pd.isna(row['address']):
        address_list=row['address'].split(', ')
        COUNTIES=[county.strip() for county in address_list if 'County' in county]
        BORO=[boro.strip() for boro in address_list if boro.strip() in boroughs]
        
            
        if COUNTIES:
            COUNTY5=[county for county in COUNTIES if county in counties_dict.keys()]
            if COUNTY5:
                return counties_dict.get(COUNTY5[0])
            else:
                return COUNTIES[0]
        elif BORO:
            return BORO[0]
        else:
            return row['address']
                
            
    else:
        return row['borough']



def extract_lat_long(row):
    if pd.isna(row['latitude']):
        latitude= 999.0
    else:
        latitude=float(row['latitude'])
    if pd.isna(row['longitude']):
        longitude= 999.0
    else:
        longitude=float(row['longitude'])
    return latitude,longitude





def capturing_boro_and_zip_code(location):
    """ReverseGeocoding and extracting borough and zip_code
        from object
    Input: geopy.location.Location
    Return: boro and zip_code as a strings
    
    """
    address_list=location.address.split(', ')
    COUNTIES=[county.strip() for county in address_list if 'County' in county]
    BORO=[boro.strip() for boro in address_list if boro.strip() in boroughs]
    zipcodes=[]
    for string in address_list:
        match = re.search(r"\b\d{5}(?:-\d{4})?\b", string)  # Matches 5-digit and 9-digit (ZIP+4) formats
        if match:
            zipcodes.append(match.group())
    if COUNTIES:
        COUNTY5=[county for county in COUNTIES if county in counties_nyc_dict.keys()]
        if COUNTY5:
            return counties_nyc_dict.get(COUNTY5[0]),zipcodes[0]
        else:
            return COUNTIES[0],zipcodes[0]
    elif BORO:
        return BORO[0],zipcodes[0]
    else:
        return None, None
    

def detect_zipcodes(row):
    pattern = r"\b\d{5}(?:-\d{4})?\b"
    if pd.isna(row['zip_code']) and not pd.isna(row['address']):
        if row['address']=='Nothing':
            return np.nan
        elif re.findall(pattern, row['address']):
            return re.findall(pattern, row['address'])[0]
        else:
            return 'XXX'
    else:
        return row['zip_code']
#retrieving ny zipcodes of counties from NYC and around
ny_counties=['Westchester', 'Rockland', 'Nassau', 'Kings', 'New York', 'Richmond','Queens', 'Bronx']
def reteieving_ny_zipcodes(ny_counties):
    zipcodesny=[]
    for county in ny_counties:
        url=f'https://data.ny.gov/resource/juva-r6g2.json?county={county}'
        counties_json=requests.get(url).json()
        for item in counties_json:
            zipcodesny.append(item['zip_code'])
    return zipcodesny

nj_counties=['bergen', 'hudson', 'union', 'middlesex']
zipcodesnj=[]
url='https://www.corragroup.com/new-jersey-county-lookup.html'
nj_df=pd.read_html(url)[0]
nj_df['County']=nj_df.County.str.lower()
zipcodesnj=nj_df[nj_df.County.isin(nj_counties)]['ZIP Code'].unique()


def retrieving_nj_zipcodes(ny_counties):
    zipcodesnj=[]
    url='https://www.corragroup.com/new-jersey-county-lookup.html'
    nj_df=pd.read_html(url)[0]
    nj_df['County']=nj_df.County.str.lower()
    zipcodesnj=list(nj_df[nj_df.County.isin(nj_counties)]['ZIP Code'].unique())
    return zipcodesny


print("DONE")

DONE


- Created a new file called new_crash where I added a column address___. My next step is to replace
- borough and zip_code where I have a ddress and they are null.

- At this point I want to replacethe file called Motor_Vehicle_... with the new one.
- Start again the code from 2024/11/30 with the new dataset.
- Add step where I check if borough and zipcode are null while not location.
- trying to create a dict where to keep all combinations of loc,borough and zip_code. Also I am collecting all zip codes from NY and NJ to verify values when uploading
- Next step: get new_df not null location if borough and zipcode are null and fill the null if possible. ALso if location not in dictionri add it as a key with borough and zip_codeIf it is the case I will frirst search in the long list of of combos I already have( make a set from new_carsh where to search) and if it fails do a reverse geocode.
- # DONE
- Try it out with all new dates up until December 31st 2024. It sshould work and I will move on.
- Must try on 1/28/25

In [28]:
#Announcing beginning and Where to start
with open('./data/pickle_dict.pkl','rb') as file:
    my_pickle=pickle.load(file)
start_date=my_pickle['start_date']
print("START DATE: ",start_date)
#Where to end:
final_date=input("Enter new date 'yyyy-mm-dd'")
#final_date='2024-12-05'
final_date_dt= datetime.strptime(final_date, '%Y-%m-%d')
start_date_dt= datetime.strptime(start_date, '%Y-%m-%d')
##Only need to do it the first tim
#start_date=start_date.to_pydatetime()
print("START DATE: ",start_date,"\n", "FINAL DATE: ", final_date)
start_date_str=str(start_date_dt)[:10]
new_date=start_date_dt+timedelta(1)
print(new_date,type(new_date), start_date_dt, final_date_dt)
new_df=pd.DataFrame()
while new_date <= final_date_dt:
    new_date_str=str(new_date)[:10]
    url=f'https://data.cityofnewyork.us/resource/h9gi-nx95.json?$$app_token={app_Token}&crash_date={new_date_str}'
    resp=requests.get(url).json()
    print('RESP LEN:',len(resp))
    if new_df.empty:
        new_df=pd.DataFrame(resp)
        print(new_df.shape[0])
        new_date=new_date+timedelta(1)
    else:
        new_df=pd.concat([new_df,pd.DataFrame(resp)])
        print(new_date,new_df.shape[0])
        new_date=new_date+timedelta(1)
#Fixing column names
rename_columns={'vehicle_type_code1':'vehicle_type_code_1',
               'vehicle_type_code2':'vehicle_type_code_2'}
new_df.rename(columns=rename_columns,inplace=True)
new_df.reset_index(inplace=True,drop=True)
#cast to string and do the call.
assert new_df.shape[1]==my_pickle['crashes'].shape[1]
new_df['crash_date']=pd.to_datetime(new_df.crash_date)
#Before concat I should reverse geo code if necessary:
#better with smaller dataset (new_df)
#new_df['location']=tuple(zip(new_df.latitude.astype(float),new_df.longitude.astype(float)))
new_df['location']=new_df.apply(extract_lat_long,axis=1)
new_df['borough']=new_df['borough'].fillna('0')
new_df['zip_code']=new_df['zip_code'].fillna('no-res')
for idx, row in new_df.iterrows():
    #print(idx,row.location,row.zip_code,row.borough)
    if row.location[0]!=999.0 and  row.borough!='no-res' and  row.zip_code!='no-res':
        #print(row.borough)
        if not my_pickle['location_dict'].get(row.location):
            my_pickle['location_dict'][row.location]={}
            my_pickle['location_dict'][row.location]['borough']=row.borough
            my_pickle['location_dict'][row.location]['zip_code']=row.zip_code
    elif row.location[0] !=999.0 and not my_pickle['location_dict'].get(row.location):
        
        #print(row.location)
        location = geolocator.reverse((row.location[0],row.location[1]), exactly_one=True)
        time.sleep(1)
        if location:
            boro, zip_=capturing_boro_and_zip_code(location)
            print(boro,zip_)
            new_df.at[idx,'borough']=boro
            new_df.at[idx,'zip_code']=zip_
            my_pickle['location_dict'][row.location]={}
            my_pickle['location_dict'][row.location]['borough']=boro
            my_pickle['location_dict'][row.location]['zip_code']=zip_
       
    elif pd.notnull(row.location) and my_pickle['location_dict'].get(row.location):
        if pd.isna(row.borough):
            print("fixing it",my_pickle['location_dict'].get(row.location)['borough'])
            new_df.at[idx,'borough']=my_pickle['location_dict'].get(row.location)['borough']
        if pd.isna(row.zip_code):
            new_df.at[idx,'zip_code']=my_pickle['location_dict'].get(row.location)['zip_code']
my_pickle['crashes']=pd.concat([my_pickle['crashes'], new_df])
my_pickle['start_date']=str(new_date-timedelta(1))[:10]
with open('./data/pickle_dict.pkl','wb') as file:
    pickle.dump(my_pickle,file)
play_done_message()
print("Done")

START DATE:  2025-02-28


Enter new date 'yyyy-mm-dd' 2025-03-31


START DATE:  2025-02-28 
 FINAL DATE:  2025-03-31
2025-03-01 00:00:00 <class 'datetime.datetime'> 2025-02-28 00:00:00 2025-03-31 00:00:00
RESP LEN: 195
195
RESP LEN: 193
2025-03-02 00:00:00 388
RESP LEN: 229
2025-03-03 00:00:00 617
RESP LEN: 195
2025-03-04 00:00:00 812
RESP LEN: 244
2025-03-05 00:00:00 1056
RESP LEN: 230
2025-03-06 00:00:00 1286
RESP LEN: 264
2025-03-07 00:00:00 1550
RESP LEN: 216
2025-03-08 00:00:00 1766
RESP LEN: 201
2025-03-09 00:00:00 1967
RESP LEN: 221
2025-03-10 00:00:00 2188
RESP LEN: 228
2025-03-11 00:00:00 2416
RESP LEN: 205
2025-03-12 00:00:00 2621
RESP LEN: 257
2025-03-13 00:00:00 2878
RESP LEN: 275
2025-03-14 00:00:00 3153
RESP LEN: 228
2025-03-15 00:00:00 3381
RESP LEN: 225
2025-03-16 00:00:00 3606
RESP LEN: 209
2025-03-17 00:00:00 3815
RESP LEN: 252
2025-03-18 00:00:00 4067
RESP LEN: 237
2025-03-19 00:00:00 4304
RESP LEN: 249
2025-03-20 00:00:00 4553
RESP LEN: 261
2025-03-21 00:00:00 4814
RESP LEN: 208
2025-03-22 00:00:00 5022
RESP LEN: 176
2025-03-23 00:

In [29]:
import pickle
with open('./data/pickle_dict.pkl','rb') as file:
    my_pickle=pickle.load(file)
my_pickle.keys()

dict_keys(['start_date', 'crashes', 'NY_zipcodes', 'NJ_zipcodes', 'location_dict'])

In [30]:
my_pickle['crashes'].sort_values('crash_date').tail()

,crash_date,crash_time,borough,zip_code,latitude,longitude,location,on_street_name,off_street_name,number_of_persons_injured,...,contributing_factor_vehicle_1,contributing_factor_vehicle_2,vehicle_type_code_1,vehicle_type_code_2,contributing_factor_vehicle_3,vehicle_type_code_3,contributing_factor_vehicle_4,vehicle_type_code_4,contributing_factor_vehicle_5,vehicle_type_code_5
6839,2025-03-31,22:03,MANHATTAN,10019,40.76284,-73.98228,"(40.76284, -73.98228)",NaN,NaN,2,...,Unspecified,NaN,Sedan,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6840,2025-03-31,8:18,0,no-res,40.715443,-73.95184,"(40.715443, -73.95184)",MEEKER AVE,UNION AVE,0,...,Brakes Defective,Other Vehicular,Flat Bed,Sedan,Unspecified,Sedan,NaN,NaN,NaN,NaN
6841,2025-03-31,10:24,BROOKLYN,11234,40.61252,-73.906746,"(40.61252, -73.906746)",WHITMAN DR,E 66 ST,1,...,Driver Inattention/Distraction,Unspecified,Sedan,Bike,NaN,NaN,NaN,NaN,NaN,NaN
6818,2025-03-31,16:50,BROOKLYN,11230,40.62657,-73.970924,"(40.62657, -73.970924)",OCEAN PKWY,AVENUE I,1,...,Driver Inattention/Distraction,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6965,2025-03-31,20:09,BROOKLYN,11208,40.66907,-73.85874,"(40.66907, -73.85874)",NaN,NaN,0,...,Passing or Lane Usage Improper,Unspecified,Sedan,Sedan,Unspecified,Sedan,NaN,NaN,NaN,NaN


In [10]:
#Announcing beginning and Where to start
with open('./data/pickle_dict.pkl','rb') as file:
    my_pickle=pickle.load(file)
start_date=my_pickle['start_date']
print("START DATE: ",start_date)
#Where to end:
final_date=input("Enter new date 'yyyy-mm-dd'")
#final_date='2024-12-05'
final_date_dt= datetime.strptime(final_date, '%Y-%m-%d')
start_date_dt= datetime.strptime(start_date, '%Y-%m-%d')
##Only need to do it the first tim
#start_date=start_date.to_pydatetime()
print("START DATE: ",start_date,"\n", "FINAL DATE: ", final_date)
start_date_str=str(start_date_dt)[:10]
new_date=start_date_dt+timedelta(1)
print(new_date,type(new_date), start_date_dt, final_date_dt)
new_df=pd.DataFrame()
while new_date <= final_date_dt:
    new_date_str=str(new_date)[:10]
    url=f'https://data.cityofnewyork.us/resource/h9gi-nx95.json?$$app_token={app_Token}&crash_date={new_date_str}'
    resp=requests.get(url).json()
    print('RESP LEN:',len(resp))
    if new_df.empty:
        new_df=pd.DataFrame(resp)
        print(new_df.shape[0])
        new_date=new_date+timedelta(1)
    else:
        new_df=pd.concat([new_df,pd.DataFrame(resp)])
        print(new_date,new_df.shape[0])
        new_date=new_date+timedelta(1)
        
rename_columns={'vehicle_type_code1':'vehicle_type_code_1',
               'vehicle_type_code2':'vehicle_type_code_2'}
new_df.rename(columns=rename_columns,inplace=True)
new_df.reset_index(inplace=True,drop=True)
new_df.shape

START DATE:  2025-02-28


Enter new date 'yyyy-mm-dd' 2025-03-31


START DATE:  2025-02-28 
 FINAL DATE:  2025-03-31
2025-03-01 00:00:00 <class 'datetime.datetime'> 2025-02-28 00:00:00 2025-03-31 00:00:00
RESP LEN: 195
195
RESP LEN: 193
2025-03-02 00:00:00 388
RESP LEN: 229
2025-03-03 00:00:00 617
RESP LEN: 195
2025-03-04 00:00:00 812
RESP LEN: 244
2025-03-05 00:00:00 1056
RESP LEN: 230
2025-03-06 00:00:00 1286
RESP LEN: 264
2025-03-07 00:00:00 1550
RESP LEN: 216
2025-03-08 00:00:00 1766
RESP LEN: 201
2025-03-09 00:00:00 1967
RESP LEN: 221
2025-03-10 00:00:00 2188
RESP LEN: 228
2025-03-11 00:00:00 2416
RESP LEN: 205
2025-03-12 00:00:00 2621
RESP LEN: 257
2025-03-13 00:00:00 2878
RESP LEN: 275
2025-03-14 00:00:00 3153
RESP LEN: 228
2025-03-15 00:00:00 3381
RESP LEN: 225
2025-03-16 00:00:00 3606
RESP LEN: 209
2025-03-17 00:00:00 3815
RESP LEN: 252
2025-03-18 00:00:00 4067
RESP LEN: 237
2025-03-19 00:00:00 4304
RESP LEN: 249
2025-03-20 00:00:00 4553
RESP LEN: 261
2025-03-21 00:00:00 4814
RESP LEN: 208
2025-03-22 00:00:00 5022
RESP LEN: 176
2025-03-23 00:

(6966, 29)

In [21]:
my_pickle['crashes'].crash_date.max(),my_pickle['crashes'].shape

(Timestamp('2025-02-28 00:00:00'), (2158889, 29))

In [27]:
new_df.shape

(6966, 29)

In [25]:
pd.concat([my_pickle['crashes'],new_df]).isnull().sum()

crash_date                             0
crash_time                             0
borough                           359613
zip_code                          359879
latitude                          239796
longitude                         239796
location                          239671
on_street_name                    466697
off_street_name                  1783465
number_of_persons_injured             18
number_of_persons_killed              31
number_of_pedestrians_injured          0
number_of_pedestrians_killed           0
number_of_cyclist_injured              0
number_of_cyclist_killed               0
number_of_motorist_injured             0
number_of_motorist_killed              0
collision_id                           0
cross_street_name                 834333
contributing_factor_vehicle_1       7461
contributing_factor_vehicle_2     343656
vehicle_type_code_1                15271
vehicle_type_code_2               427440
contributing_factor_vehicle_3    2009600
vehicle_type_cod

In [19]:
my_pickle['crashes'].isnull().sum()

crash_date                             0
crash_time                             0
borough                           358078
zip_code                          358344
latitude                          239695
longitude                         239695
location                          239570
on_street_name                    464614
off_street_name                  1780621
number_of_persons_injured             18
number_of_persons_killed              31
number_of_pedestrians_injured          0
number_of_pedestrians_killed           0
number_of_cyclist_injured              0
number_of_cyclist_killed               0
number_of_motorist_injured             0
number_of_motorist_killed              0
collision_id                           0
cross_street_name                 829449
contributing_factor_vehicle_1       7405
contributing_factor_vehicle_2     341928
vehicle_type_code_1                15141
vehicle_type_code_2               425027
contributing_factor_vehicle_3    2003236
vehicle_type_cod

In [39]:
rename_columns={'vehicle_type_code1':'vehicle_type_code_1',
               'vehicle_type_code2':'vehicle_type_code_2'}
new_df.rename(columns=rename_columns,inplace=True)
new_df.reset_index(inplace=True,drop=True)

Index(['crash_date', 'crash_time', 'borough', 'zip_code', 'latitude',
       'longitude', 'location', 'on_street_name', 'off_street_name',
       'number_of_persons_injured', 'number_of_persons_killed',
       'number_of_pedestrians_injured', 'number_of_pedestrians_killed',
       'number_of_cyclist_injured', 'number_of_cyclist_killed',
       'number_of_motorist_injured', 'number_of_motorist_killed',
       'collision_id', 'cross_street_name', 'contributing_factor_vehicle_1',
       'contributing_factor_vehicle_2', 'vehicle_type_code_1',
       'vehicle_type_code_2', 'contributing_factor_vehicle_3',
       'vehicle_type_code_3', 'contributing_factor_vehicle_4',
       'vehicle_type_code_4', 'contributing_factor_vehicle_5',
       'vehicle_type_code_5'],
      dtype='object')

In [14]:
import pandas as pd
df=pd.read_csv('./data/crashesupto2024.csv')
df.head()

/var/folders/ky/93l9lrgs7qgbn3j0hprgld5m0000gn/T/ipykernel_2876/2695577671.py:2: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  df=pd.read_csv('./data/crashesupto2024.csv')


,crash_date,crash_time,borough,zip_code,latitude,longitude,location,on_street_name,cross_street_name,off_street_name,...,collision_id,vehicle_type_code_1,vehicle_type_code_2,vehicle_type_code_3,vehicle_type_code_4,vehicle_type_code_5,hour,weekday,month,year
0,2021-09-11,2:39,NaN,NaN,NaN,NaN,NaN,WHITESTONE EXPRESSWAY,20 AVENUE,NaN,...,4455765,Sedan,Sedan,NaN,NaN,NaN,2,Sat,Sep,2021
1,2022-03-26,11:45,NaN,NaN,NaN,NaN,NaN,QUEENSBORO BRIDGE UPPER,NaN,NaN,...,4513547,Sedan,NaN,NaN,NaN,NaN,11,Sat,Mar,2022
2,2022-06-29,6:55,NaN,NaN,NaN,NaN,NaN,THROGS NECK BRIDGE,NaN,NaN,...,4541903,Sedan,Pick-up Truck,NaN,NaN,NaN,6,Wed,Jun,2022
3,2021-09-11,9:35,BROOKLYN,11208.0,40.667202,-73.866500,"(40.667202, -73.8665)",NaN,NaN,1211 LORING AVENUE,...,4456314,Sedan,NaN,NaN,NaN,NaN,9,Sat,Sep,2021
4,2021-12-14,8:13,BROOKLYN,11233.0,40.683304,-73.917274,"(40.683304, -73.917274)",SARATOGA AVENUE,DECATUR STREET,NaN,...,4486609,NaN,NaN,NaN,NaN,NaN,8,Tue,Dec,2021


In [26]:
with open('./data/pickle_dict.pkl','rb') as file:
    my_pickle=pickle.load(file)
start_date=my_pickle['start_date']

In [15]:
df.crash_date.max(),df.crash_date.min()

('2024-12-31', '2013-01-01')

In [52]:
new_df.shape[0],my_pickle['crashes'].shape[0]

2158889

In [43]:
final_df=pd.concat([new_df,my_pickle['crashes']])

In [40]:
set(my_pickle['crashes'].columns)-set(new_df.columns)

set()

In [60]:
final_df['crash_date']=pd.to_datetime(final_df.crash_date)

In [62]:
final_df.crash_date.max()

Timestamp('2025-02-28 00:00:00')

In [63]:
my_pickle.keys()

dict_keys(['start_date', 'crashes', 'NY_zipcodes', 'NJ_zipcodes', 'location_dict'])

In [65]:
my_pickle['start_date']='2025-02-28'

In [66]:
my_pickle['crashes']=final_df

In [69]:
my_pickle['crashes'].crash_date.max()

Timestamp('2025-02-28 00:00:00')

In [70]:
with open('./data/pickle_dict.pkl','wb') as file:
    pickle.dump(my_pickle,file)